In [3]:
import numpy as np 
import pandas as pd 
import csv 
import os 

import torch 


torch.set_printoptions(edgeitems=2, precision=2, linewidth=75)

In [4]:
path = os.getcwd()
DATA_PATH = "/".join(p for p in path.split(os.path.sep)[:-1]) + "/data/p1ch4/"
CSV_PATH = DATA_PATH + "tabular-wine/winequality-white.csv"

In [5]:
wineq_numpy = np.loadtxt(CSV_PATH, dtype=np.float32, delimiter=';', skiprows=1)

wineq_numpy

array([[ 7.  ,  0.27,  0.36, ...,  0.45,  8.8 ,  6.  ],
       [ 6.3 ,  0.3 ,  0.34, ...,  0.49,  9.5 ,  6.  ],
       [ 8.1 ,  0.28,  0.4 , ...,  0.44, 10.1 ,  6.  ],
       ...,
       [ 6.5 ,  0.24,  0.19, ...,  0.46,  9.4 ,  6.  ],
       [ 5.5 ,  0.29,  0.3 , ...,  0.38, 12.8 ,  7.  ],
       [ 6.  ,  0.21,  0.38, ...,  0.32, 11.8 ,  6.  ]], dtype=float32)

In [6]:
col_list = next(csv.reader(open(CSV_PATH), delimiter=';'))

wineq_numpy.shape, col_list

((4898, 12),
 ['fixed acidity',
  'volatile acidity',
  'citric acid',
  'residual sugar',
  'chlorides',
  'free sulfur dioxide',
  'total sulfur dioxide',
  'density',
  'pH',
  'sulphates',
  'alcohol',
  'quality'])

In [7]:
wineq = torch.from_numpy(wineq_numpy)

wineq

tensor([[ 7.00,  0.27,  ...,  8.80,  6.00],
        [ 6.30,  0.30,  ...,  9.50,  6.00],
        ...,
        [ 5.50,  0.29,  ..., 12.80,  7.00],
        [ 6.00,  0.21,  ..., 11.80,  6.00]])

In [10]:
data = wineq[:, :-1]
data, data.shape

(tensor([[ 7.00,  0.27,  ...,  0.45,  8.80],
         [ 6.30,  0.30,  ...,  0.49,  9.50],
         ...,
         [ 5.50,  0.29,  ...,  0.38, 12.80],
         [ 6.00,  0.21,  ...,  0.32, 11.80]]),
 torch.Size([4898, 11]))

In [11]:
target = wineq[:, -1].long() # quality

target, target.shape

(tensor([6, 6,  ..., 7, 6]), torch.Size([4898]))

# When Categorize
<img src="https://github.com/hoangminhtoan/2020_Projects/blob/master/Python_DL/PytorchNLP/data/images/how_to_treat_data.png">


In [12]:
target_onehot = torch.zeros(target.shape[0], 10)

target_onehot.scatter_(1, target.unsqueeze(1), 1.0)

target_onehot, target_onehot.shape

(tensor([[0., 0.,  ..., 0., 0.],
         [0., 0.,  ..., 0., 0.],
         ...,
         [0., 0.,  ..., 0., 0.],
         [0., 0.,  ..., 0., 0.]]),
 torch.Size([4898, 10]))

In [17]:
data_mean = torch.mean(data, dim=0)

data_mean, data_mean.shape

(tensor([6.85e+00, 2.78e-01, 3.34e-01, 6.39e+00, 4.58e-02, 3.53e+01,
         1.38e+02, 9.94e-01, 3.19e+00, 4.90e-01, 1.05e+01]),
 torch.Size([11]))

In [18]:
data_var = torch.var(data, dim=0)

data_var, data_var.shape

(tensor([7.12e-01, 1.02e-02, 1.46e-02, 2.57e+01, 4.77e-04, 2.89e+02,
         1.81e+03, 8.95e-06, 2.28e-02, 1.30e-02, 1.51e+00]),
 torch.Size([11]))